***Do not re-run this code***


In [1]:
%%capture
pip install datasets

In [2]:
from datasets import list_datasets, load_dataset

In [12]:
is_train_valid_test = True#@param{type: "boolean"}
if is_train_valid_test:
    dataset = load_dataset('hatexplain', split = ['train', 'validation', 'test'])

Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/802fcd855438812094e336cea509c99b04b890e4e0846c0385877ee2c7361e93)


In [13]:
if is_train_valid_test:
    train_dataset = dataset[0]
    valid_dataset = dataset[1]
    test_dataset = dataset[2]

In [14]:
import json
if is_train_valid_test:
    print(len(train_dataset), len(valid_dataset), len(test_dataset))
    print(json.dumps(train_dataset[0], indent = 2))    
else:
    print(len(dataset))
    print(json.dumps(dataset[0], indent = 2))

15383 1922 1924
{
  "annotators": {
    "annotator_id": [
      203,
      204,
      233
    ],
    "label": [
      0,
      2,
      2
    ],
    "target": [
      [
        "Hindu",
        "Islam"
      ],
      [
        "Hindu",
        "Islam"
      ],
      [
        "Hindu",
        "Islam",
        "Other"
      ]
    ]
  },
  "id": "23107796_gab",
  "post_tokens": [
    "u",
    "really",
    "think",
    "i",
    "would",
    "not",
    "have",
    "been",
    "raped",
    "by",
    "feral",
    "hindu",
    "or",
    "muslim",
    "back",
    "in",
    "india",
    "or",
    "bangladesh",
    "and",
    "a",
    "neo",
    "nazi",
    "would",
    "rape",
    "me",
    "as",
    "well",
    "just",
    "to",
    "see",
    "me",
    "cry"
  ],
  "rationales": [
    [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      1,
      1,
 

In [15]:
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/AI&Ethics/'
os.chdir(root_path)

annotator_file = "data/annotators_demography.csv"

Mounted at /content/gdrive



***Can re-run code starting from the next block***


In [16]:
def get_ids(criteria):
    if criteria == 'Race':
        column = 2
    elif criteria == 'Country':
        column = 3
    elif criteria == 'Age':
        column = 4
    elif criteria == 'Gender':
        column = 5
    elif criteria == 'Religion':
        column = 6
    
    ids = {}
    with open(annotator_file) as annotfile:
        next(annotfile)
        annots = annotfile.read().splitlines()
        for a in annots:
            annot = a.split(",")
            try:
                ids[annot[column]].append(annot[1])
            except KeyError:
                ids[annot[column]] = [annot[1]]
    return ids

def get_freq(id, ids):
    for cl in ids.keys():
        if str(id) in ids[cl]:
            return len(ids[cl])
    return 200

def get_annotator_class(id, ids):
    for cl in ids.keys():
        if str(id) in ids[cl]:
            return cl
    return None

ids = get_ids('Gender')
print(ids)

{'Female': ['106', '219', '114', '118', '142', '75', '132', '135', '13', '63', '166', '82', '183', '134', '120', '62', '48', '187', '130', '89', '148', '167', '113', '143', '102', '55', '93', '73', '70', '121', '72', '28', '140', '168', '85', '92', '179', '14', '136', '74', '169', '137', '95', '76', '180', '104', '94', '123', '162', '107', '188', '198', '189', '44', '100', '38', '84', '133', '116', '1', '96', '192', '2', '32', '156', '197', '83', '36', '171', '127', '154', '71', '3', '45', '79', '49', '97', '77', '177', '170', '176', '240', '199', '239', '222', '210', '182', '213', '249', '220', '230', '25', '252', '235', '212', '211', '251', '237', '242', '247', '244', '232', '234', '231', '204'], 'Prefer not to disclose': ['18'], 'Male': ['8', '173', '9', '58', '164', '178', '16', '125', '65', '109', '50', '146', '34', '101', '52', '194', '98', '66', '149', '111', '7', '23', '68', '139', '124', '80', '37', '35', '108', '10', '5', '59', '128', '144', '147', '110', '21', '153', '209', 

In [17]:
import pandas as pd

df_annot_ = pd.read_csv(annotator_file)

In [19]:
df_annot_.head()

,Unnamed: 0,id,Race,Country,Age,Gender,Religion,Freedom_of_speech,Victim
0,0,106,Caucasian,USA,40--49,Female,Christian,No,No
1,1,18,Hispanic,Brazil,30--39,Prefer not to disclose,Other,No,Yes
2,2,219,Caucasian,UK,30--39,Female,Nonreligious,No,Prefer not to disclose
3,3,8,Caucasian,USA,30--39,Male,Christian,Yes,No
4,4,173,Caucasian,USA,65++,Male,Christian,Yes,Prefer not to disclose


In [21]:
# demographic annotators' information in the form of a dictionary, key being annotator id, and value being another dictionary containing demographic info on the annotator

annot_info_dict = {}
for i in range(df_annot_.shape[0]):
    id_ = df_annot_['id'][i]
    annot_info_dict[id_] = {}
    for col in ["Race",	"Country",	"Age",	"Gender",	"Religion"]:
        annot_info_dict[id_][col] = df_annot_[col][i]

In [36]:
import random

# this block represents randomly picking annotator - for annotator demography prediction

def save_dataset(dataset, annot_info_dict, save_path):

    random.seed(0)

    target_dataset = []
    for i in range(len(dataset)):
        # print(i)
        annotators = dataset[i]['annotators']
        num_annotators = len(annotators['annotator_id'])
        index = random.randint(0, num_annotators - 1)
        if annotators['annotator_id'][index] in annot_info_dict:
            annot_dataset = annot_info_dict[annotators['annotator_id'][index]]
        else:
            continue # skipping those sampled annotators not present in demographics csv.

        label_dataset = {
            "label": annotators['label'][index], 
            "annotator_id": annotators['annotator_id'][index], 
            "target": annotators['target'][index],
            'text': dataset[i]['post_tokens']
            }
        
        # merging the label and annotator information dictionaries

        result_dataset = {**label_dataset, **annot_dataset}

        target_dataset.append(result_dataset)

    print("Number of samples: {}".format(len(target_dataset)))
    # print(target_dataset[0])

    with open(save_path, 'w') as f:
        json.dump(target_dataset, f, indent = 2)

if is_train_valid_test:
    save_dataset(train_dataset, annot_info_dict, "data/train_demographic.json")
    save_dataset(valid_dataset, annot_info_dict, "data/valid_demographic.json")
    save_dataset(test_dataset, annot_info_dict, "data/test_demographic.json")

Number of samples: 11553
Number of samples: 1419
Number of samples: 1407


In [ ]:
# this block represents the non-greedy approach, where we choose the least represented class at each step (class representation is not updated dynamically, instead chosen from overall distribution)
target_dataset = []
for i in range(len(dataset)):
    annotators = dataset[i]['annotators']
    dominant_label = max(set(annotators['label']), key = annotators['label'].count) 
    if annotators['label'].count(dominant_label) != 1:
        min_freq = 200 #arbitrary number higher than the number of annotators
        for annotator in annotators['annotator_id']:
            # print(min_freq)
            index = annotators['annotator_id'].index(annotator)
            # print(get_freq(annotator, ids))
            if annotators['label'][index] == dominant_label and get_freq(annotator, ids) <= min_freq:
                min_freq = get_freq(annotator, ids)
                label_dataset = {"label": annotators['label'][index], "annotator_id": annotator, "target": annotators['target'][index]}
            else:
                continue
        target_dataset.append(label_dataset)
    else:
        continue
    # print(annotators)
    # print(dominant_label)
    # print(label_dataset)

print(target_dataset)

with open(os.path.join(data_dir_path, "non_greedy_approach_annotators.json"), 'w') as f:
    json.dump(target_dataset, f, indent = 2)

[{'label': 2, 'annotator_id': 204, 'target': ['Hindu', 'Islam']}, {'label': 2, 'annotator_id': 6, 'target': ['Indian']}, {'label': 2, 'annotator_id': 209, 'target': ['Other']}, {'label': 2, 'annotator_id': 209, 'target': ['Hindu']}, {'label': 2, 'annotator_id': 25, 'target': ['Economic']}, {'label': 1, 'annotator_id': 204, 'target': ['Hispanic', 'Refugee']}, {'label': 0, 'annotator_id': 4, 'target': ['Islam', 'Women', 'Homosexual']}, {'label': 1, 'annotator_id': 204, 'target': ['Arab', 'Hindu', 'Islam']}, {'label': 1, 'annotator_id': 211, 'target': ['None']}, {'label': 0, 'annotator_id': 228, 'target': ['Refugee']}, {'label': 1, 'annotator_id': 221, 'target': ['Caucasian', 'Men', 'Women']}, {'label': 1, 'annotator_id': 204, 'target': ['Caucasian']}, {'label': 1, 'annotator_id': 199, 'target': ['Caucasian']}, {'label': 1, 'annotator_id': 233, 'target': ['Caucasian']}, {'label': 0, 'annotator_id': 223, 'target': ['Refugee']}, {'label': 1, 'annotator_id': 18, 'target': ['None']}, {'label'

NameError: ignored

In [ ]:
# this block represents the greedy approach, where we choose the least represented class at each step (class representation is updated dynamically)
import random

annotator_classwise_freq = {}
for i in ids.keys():
    annotator_classwise_freq[i] = 0

c1 = 0

target_dataset = []
for i in range(len(dataset)):
    # print(annotator_classwise_freq)
    annotators = dataset[i]['annotators']
    dominant_label = max(set(annotators['label']), key = annotators['label'].count) 
    if annotators['label'].count(dominant_label) != 1:        
        temp = {}
        for annotator in annotators['annotator_id']:
            index = annotators['annotator_id'].index(annotator)
            annotator_class = get_annotator_class(annotator, ids)
            if annotators['label'][index] == dominant_label and annotator_class != None:
                temp[annotator] = annotator_classwise_freq[annotator_class]
        if len(temp.keys()) != 0:    
            # randomly choose one annotator having lowest representation till that iteration  
            min_value = min(list(temp.values()))
            min_keys = [k for k in temp if temp[k] == min_value]
            chosen_annotator = random.choice(min_keys)
            index = annotators['annotator_id'].index(chosen_annotator)
            annotator_classwise_freq[get_annotator_class(chosen_annotator, ids)] = annotator_classwise_freq[get_annotator_class(chosen_annotator, ids)] + 1
            # print(dataset[i]['post_tokens'])
            label_dataset = {"label": annotators['label'][index], "annotator_id": chosen_annotator, "target": annotators['target'][index], 'text': dataset[i]['post_tokens']}
            target_dataset.append(label_dataset)
        else:
            c1 = c1 + 1
            # pass
            # this represents cases where both annotators in majority class are not found in demographics
            # print(annotators)
    else:
        continue

print(c1)
print(annotator_classwise_freq)
print(target_dataset)
import json
with open('./data/greedy_approach_annotators_gender.json', 'w') as f:
    json.dump(target_dataset, f, indent = 2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

